In [ ]:
# Install necessary libraries
!pip install rasterio geopandas scikit-learn matplotlib seaborn

In [ ]:
# Import necessary libraries
import rasterio
from rasterio.plot import show
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
import seaborn as sns
from sklearn.cluster import KMeans
from sklearn.metrics import confusion_matrix
import geopandas as gpd

# Step 1: Read the multiband tiff image (import in Colab or from your Google Drive folder)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
image_path = '/content/drive/MyDrive/Colab Notebooks/2018_12_7_rgb_nir.tif'  # Update with the correct file path
image = rasterio.open(image_path)

# Visualize True Color image
# Read the band values into numpy arrays
red = image.read(3)
green = image.read(2)
blue = image.read(1)

# Function to normalize the grid values
def normalize(array):
    """Normalizes numpy arrays into scale 0.0 - 1.0"""
    array_min, array_max = array.min(), array.max()
    return ((array - array_min)/(array_max - array_min))

# Normalize the bands
redn = normalize(red)
greenn = normalize(green)
bluen = normalize(blue)

rgb = np.dstack((redn, greenn, bluen))

plt.imshow(rgb)

# Step 2: Preprocess the image by adjusting the array shape for the classifier

In [ ]:
image_data = image.read().transpose(1, 2, 0)  # Move bands to the last axis
rows, cols, bands = image_data.shape
image_data_reshaped = image_data.reshape(rows * cols, bands)

# Step 3: Perform K-means clustering

In [ ]:
num_classes = 2  # Number of classes (clusters)
kmeans = KMeans(n_clusters=num_classes, random_state=42)
kmeans.fit(image_data_reshaped)
predicted_classes = kmeans.labels_.reshape(rows, cols)

# Step 4: Visualize the classified image

In [ ]:
colors = ['white', 'black'] # One color for each clusters
class_bins = [0, 1]
cmap = ListedColormap(colors)

show(predicted_classes, cmap=cmap)

# Step 5: Save the classified image

In [ ]:
output_image_path = '/content/drive/MyDrive/Colab Notebooks/unsup_classified_image.tif'
with rasterio.open(output_image_path, 'w', **image.meta) as dst:
    dst.write(predicted_classes.astype(rasterio.uint8), 1)